In [24]:
import os
import json
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import numpy as np

# === 绘图样式设置 ===
plt.rcParams['figure.figsize'] = (32, 10)
plt.rcParams['xtick.major.width'] = 2
plt.rcParams['ytick.major.width'] = 2
plt.rcParams['axes.grid'] = False
plt.rcParams['grid.linestyle'] = '-'
plt.rcParams['grid.linewidth'] = 1
plt.rcParams['grid.color'] = '#e1e1e1'
plt.rcParams['axes.linewidth'] = 2
plt.rcParams['ytick.major.size'] = 12
plt.rcParams['xtick.major.size'] = 12
plt.rcParams['axes.titlesize'] = 52
plt.rcParams['axes.labelsize'] = 52
plt.rcParams['lines.linewidth'] = 8
plt.rcParams['lines.markersize'] = 30
plt.rcParams['xtick.labelsize'] = 52
plt.rcParams['ytick.labelsize'] = 52
plt.rcParams['font.family'] = 'serif'
plt.rcParams['font.serif'] = ['DejaVu Serif']
plt.rcParams['font.weight'] = 'normal'
plt.rcParams['axes.labelweight'] = 'normal'
plt.rcParams['axes.titleweight'] = 'normal'
plt.rcParams['axes.grid.axis'] = 'both'
plt.rcParams['axes.grid.which'] = 'major'
plt.rcParams['figure.dpi'] = 600
plt.rcParams['legend.fontsize'] = 42

# === 要处理的 JSON 文件名列表 ===
file_list = [
    "round1_fcfs_chunked_prompts_10000_qps_inf.json",
    "round1_fcfs_prompts_10000_qps_inf.json",
    "round1_ldf_rej_ada1.35_pred_prompts_10000_qps_inf.json",
    "round1_ssjf_pred_prompts_10000_qps_inf.json",
]


# 时间 bucket 间隔
bucket_interval = '30s'

# 处理每个 JSON 文件并绘图
for filename in file_list:
    policy_name = filename.split("_prompts")[0].replace("round1_", "")

    with open(filename, "r") as f:
        data = json.load(f)

    records = data["finegrained_slo_adherence"]
    df = pd.DataFrame(records, columns=["timestamp", "slo_met"])
    df["timestamp"] = pd.to_datetime(df["timestamp"])
    df.set_index("timestamp", inplace=True)

    # 1s bucket 统计
    bucketed = df.resample(bucket_interval).agg(
        slo_met_count=("slo_met", "sum"),
        total_requests=("slo_met", "count")
    )
    bucketed["slo_ratio"] = bucketed["slo_met_count"] / bucketed["total_requests"]

    # 只取前两分钟
    time_origin = bucketed.index.min()
    end_time = time_origin + pd.Timedelta(minutes=60)
    bucketed_subset = bucketed.loc[time_origin:end_time]
    relative_minutes = (bucketed_subset.index - time_origin).total_seconds() / 60

    # 绘图
    plt.figure()
    plt.plot(relative_minutes, bucketed_subset["slo_met_count"], label="# SLO met requests", linewidth=2)

    plt.xlabel("Timeline (s)")
    plt.ylabel("# SLO Met Requests")
    # plt.title(f"{policy_name} - SLO Met Requests per {bucket_interval} Bucket")

    # 设置x轴刻度
    plt.ylim(0, 400)
    xticks = np.arange(0, 2.1, 0.5)  # 每15秒一个刻度
    plt.xticks(xticks)

    # plt.grid(True)
    plt.tight_layout()
    # plt.legend()
    plt.savefig(f"{policy_name}_slo_met.pdf", dpi=600, bbox_inches='tight', format='pdf')
    plt.close()


In [6]:
import os
import json
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# === 绘图样式设置 ===
plt.rcParams['figure.figsize'] = (30, 20)
plt.rcParams['xtick.major.width'] = 2
plt.rcParams['ytick.major.width'] = 2
plt.rcParams['axes.grid'] = False
plt.rcParams['grid.linestyle'] = '-'
plt.rcParams['grid.linewidth'] = 1
plt.rcParams['grid.color'] = '#e1e1e1'
plt.rcParams['axes.linewidth'] = 2
plt.rcParams['ytick.major.size'] = 12
plt.rcParams['xtick.major.size'] = 12
plt.rcParams['axes.titlesize'] = 52
plt.rcParams['axes.labelsize'] = 52
plt.rcParams['lines.linewidth'] = 8
plt.rcParams['lines.markersize'] = 30
plt.rcParams['xtick.labelsize'] = 52
plt.rcParams['ytick.labelsize'] = 52
plt.rcParams['font.family'] = 'serif'
plt.rcParams['font.serif'] = ['DejaVu Serif']
plt.rcParams['font.weight'] = 'normal'
plt.rcParams['axes.labelweight'] = 'normal'
plt.rcParams['axes.titleweight'] = 'normal'
plt.rcParams['axes.grid.axis'] = 'both'
plt.rcParams['axes.grid.which'] = 'major'
plt.rcParams['figure.dpi'] = 600
plt.rcParams['legend.fontsize'] = 42

# === 要处理的 JSON 文件名列表 ===
file_list = [
    "round1_fcfs_chunked_prompts_10000_qps_inf.json",
    "round1_fcfs_prompts_10000_qps_inf.json",
    "round1_ldf_rej_ada1.35_pred_prompts_10000_qps_inf.json",
    "round1_ssjf_pred_prompts_10000_qps_inf.json",
]

# 时间 bucket 间隔
bucket_interval = '1s'

# 处理每个 JSON 文件并绘图
for filename in file_list:
    policy_name = filename.split("_prompts")[0].replace("round1_", "")

    with open(filename, "r") as f:
        data = json.load(f)

    records = data["finegrained_slo_adherence"]
    df = pd.DataFrame(records, columns=["timestamp", "slo_met"])
    df["timestamp"] = pd.to_datetime(df["timestamp"])
    df.set_index("timestamp", inplace=True)

    # 每秒统计总请求数和满足 SLO 的数量
    bucketed = df.resample(bucket_interval).agg(
        slo_met_count=("slo_met", "sum"),
        total_requests=("slo_met", "count")
    )
    bucketed["slo_ratio"] = bucketed["slo_met_count"] / bucketed["total_requests"]

    # === ✅ 不再截取前两分钟，而是画完整时间 ===
    time_origin = bucketed.index.min()
    relative_minutes = (bucketed.index - time_origin).total_seconds() / 60

    # 绘图
    plt.figure()
    plt.plot(relative_minutes, bucketed["slo_met_count"],
             label="SLO met per bucket", linewidth=2)
    plt.plot(relative_minutes, bucketed["total_requests"],
             label="Total requests per bucket", linewidth=2, linestyle='--')

    plt.xlabel("Time (minutes since start)")
    plt.ylabel("Requests")
    # plt.title(f"{policy_name} - SLO Met & Total Requests")

    # x轴刻度自动生成
    max_min = relative_minutes.max()
    xticks = np.arange(0, max_min + 1, 2 if max_min > 20 else 1)
    plt.xticks(xticks)

    plt.ylim(0, bucketed["total_requests"].max() * 1.1)
    plt.tight_layout()
    plt.legend()
    plt.savefig(f"{policy_name}_slo_met_and_total_fulltime.pdf", dpi=300)
    plt.close()


In [2]:
import os
import json
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import numpy as np

# === 绘图样式设置 ===
plt.rcParams['figure.figsize'] = (15, 10)
plt.rcParams['xtick.major.width'] = 2
plt.rcParams['ytick.major.width'] = 2
plt.rcParams['axes.grid'] = False
plt.rcParams['grid.linestyle'] = '-'
plt.rcParams['grid.linewidth'] = 1
plt.rcParams['grid.color'] = '#e1e1e1'
plt.rcParams['axes.linewidth'] = 2
plt.rcParams['ytick.major.size'] = 12
plt.rcParams['xtick.major.size'] = 12
plt.rcParams['axes.titlesize'] = 52
plt.rcParams['axes.labelsize'] = 52
plt.rcParams['lines.linewidth'] = 8
plt.rcParams['lines.markersize'] = 30
plt.rcParams['xtick.labelsize'] = 52
plt.rcParams['ytick.labelsize'] = 52
plt.rcParams['font.family'] = 'serif'
plt.rcParams['font.serif'] = ['DejaVu Serif']
plt.rcParams['font.weight'] = 'normal'
plt.rcParams['axes.labelweight'] = 'normal'
plt.rcParams['axes.titleweight'] = 'normal'
plt.rcParams['axes.grid.axis'] = 'both'
plt.rcParams['axes.grid.which'] = 'major'
plt.rcParams['figure.dpi'] = 600
plt.rcParams['legend.fontsize'] = 42

# === 要处理的 JSON 文件名列表 ===
file_list = [
    "r1_fcfs_mc_qps_inf.json",
    "r1_fcfs_qps_inf.json",
    "r1_ldf_rej_gate1.3_credit_qps_inf.json",
    "r1_ssjf_pred_qps_inf.json",
]


# 时间 bucket 间隔
bucket_interval = '10s'

# 处理每个 JSON 文件并绘制 SLO Ratio 图
for filename in file_list:
    policy_name = filename.split("_prompts")[0].replace("round1_", "")

    with open(filename, "r") as f:
        data = json.load(f)

    records = data["trace_slo_adherence"]
    df = pd.DataFrame(records, columns=["timestamp", "slo_met"])
    df["timestamp"] = pd.to_datetime(df["timestamp"])
    df.set_index("timestamp", inplace=True)

    print(len(df))

    # 1s bucket 统计
    bucketed = df.resample(bucket_interval).agg(
        slo_met_count=("slo_met", "sum"),
        total_requests=("slo_met", "count")
    )
    bucketed["slo_ratio"] = bucketed["slo_met_count"] / bucketed["total_requests"]

    # 只取前两分钟（或自定义分钟数）
    time_origin = bucketed.index.min()
    end_time = time_origin + pd.Timedelta(minutes=10)
    bucketed_subset = bucketed.loc[time_origin:end_time]
    relative_minutes = (bucketed_subset.index - time_origin).total_seconds() / 60

    # 绘制 SLO met ratio 图
    plt.figure()
    plt.plot(relative_minutes, bucketed_subset["slo_ratio"], label="SLO met ratio", linewidth=2)

    plt.xlabel("Timeline (min)")
    plt.ylabel("SLO Met Ratio")
    plt.ylim(-0.05, 1.05)
    
    # xticks = np.arange(0, 2., 0.5)
    # plt.xticks(xticks)

    plt.tight_layout()
    plt.savefig(f"{policy_name}_slo_met_ratio.pdf", dpi=600, bbox_inches='tight', format='pdf')
    plt.close()


3621
3621
3621
3621


In [3]:
highlight_filename = "r1_ldf_rej_gate1.3_credit_qps_inf.json"

# 预处理 ldf_rej_ada1.35_pred 的数据一次，后续重复使用
with open(highlight_filename, "r") as f:
    highlight_data = json.load(f)

highlight_df = pd.DataFrame(highlight_data["trace_slo_adherence"], columns=["timestamp", "slo_met"])
highlight_df["timestamp"] = pd.to_datetime(highlight_df["timestamp"])
highlight_df.set_index("timestamp", inplace=True)
highlight_bucketed = highlight_df.resample(bucket_interval).agg(
    slo_met_count=("slo_met", "sum"),
    total_requests=("slo_met", "count")
)
highlight_bucketed["slo_ratio"] = highlight_bucketed["slo_met_count"] / highlight_bucketed["total_requests"]
highlight_origin = highlight_bucketed.index.min()
highlight_end_time = highlight_origin + pd.Timedelta(minutes=15)
highlight_subset = highlight_bucketed.loc[highlight_origin:highlight_end_time]
highlight_minutes = (highlight_subset.index - highlight_origin).total_seconds() / 60

# 主循环绘图
for filename in file_list:
    policy_name = filename.split("_prompts")[0].replace("round1_", "")

    with open(filename, "r") as f:
        data = json.load(f)

    df = pd.DataFrame(data["trace_slo_adherence"], columns=["timestamp", "slo_met"])
    df["timestamp"] = pd.to_datetime(df["timestamp"])
    df.set_index("timestamp", inplace=True)

    bucketed = df.resample(bucket_interval).agg(
        slo_met_count=("slo_met", "sum"),
        total_requests=("slo_met", "count")
    )
    bucketed["slo_ratio"] = bucketed["slo_met_count"] / bucketed["total_requests"]
    time_origin = bucketed.index.min()
    end_time = time_origin + pd.Timedelta(minutes=15)
    bucketed_subset = bucketed.loc[time_origin:end_time]
    relative_minutes = (bucketed_subset.index - time_origin).total_seconds() / 60

    plt.figure()

    # 当前策略线
    plt.plot(relative_minutes, bucketed_subset["slo_met_count"],
             label=f"{policy_name}", linewidth=6, color="#836ed5")

    # 特殊 highlight 策略线（统一放在每张图上）
    if policy_name != "ldf_rej_ada1.35_pred":
        plt.plot(highlight_minutes, highlight_subset["slo_met_count"],
                 label="ldf_rej_ada1.35_pred", linestyle="--", linewidth=6, color="#ec6446")

    plt.xlabel("Timeline (min)")
    plt.ylabel("# SLO Met Requests")
    plt.ylim(0, 150)
    # plt.xticks(np.arange(0, 2.1, 0.5))
    # plt.xticks(np.arange(0, 2.1, 0.5))
    # plt.legend()
    plt.tight_layout()
    plt.savefig(f"{policy_name}_slo_met.pdf", dpi=600, bbox_inches='tight', format='pdf')
    plt.close()


In [3]:
import os
import json
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# === 样式设置 ===
plt.rcParams['figure.figsize'] = (15, 10)
plt.rcParams['axes.linewidth'] = 2
plt.rcParams['axes.titlesize'] = 52
plt.rcParams['axes.labelsize'] = 52
plt.rcParams['lines.linewidth'] = 8
plt.rcParams['xtick.labelsize'] = 52
plt.rcParams['ytick.labelsize'] = 52
plt.rcParams['font.family'] = 'serif'
plt.rcParams['legend.fontsize'] = 42
plt.rcParams['figure.dpi'] = 600

# === JSON 文件列表 ===
file_list = [
    "r1_fcfs_mc_qps_inf.json",
    "r1_fcfs_qps_inf.json",
    "r1_ldf_rej_gate1.3_credit_qps_inf.json",
    "r1_ssjf_pred_qps_inf.json",
]

# 时间 bucket 间隔
bucket_interval = '10s'

# 绘图
plt.figure()

for filename in file_list:
    policy_name = filename.split("_prompts")[0].replace("round1_", "")

    with open(filename, "r") as f:
        data = json.load(f)

    records = data["trace_slo_adherence"]
    df = pd.DataFrame(records, columns=["timestamp", "slo_met"])
    df["timestamp"] = pd.to_datetime(df["timestamp"])
    df.set_index("timestamp", inplace=True)

    # 时间分桶统计 slo_met 数
    bucketed = df.resample(bucket_interval).agg(slo_met_count=("slo_met", "sum"))
    bucketed = bucketed.fillna(0)

    # 累积求和
    bucketed["cumulative_slo_met"] = bucketed["slo_met_count"].cumsum()

    # 构造时间轴（分钟）
    time_origin = bucketed.index.min()
    relative_minutes = (bucketed.index - time_origin).total_seconds() / 60

    # 归一化累计值作为 CDF 曲线
    cdf_vals = bucketed["cumulative_slo_met"] / bucketed["cumulative_slo_met"].max()

    plt.plot(relative_minutes, cdf_vals, label=policy_name)

# 图标设置
plt.xlabel("Timeline (min)")
plt.ylabel("Cumulative SLO Met (Normalized)")
plt.ylim(-0.05, 1.05)
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.savefig("slo_met_over_time_cdf.pdf", dpi=600, bbox_inches='tight', format='pdf')
plt.close()


In [3]:
import os
import json
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.lines as mlines

# === 样式设置 ===
plt.rcParams['figure.figsize'] = (10, 10)
plt.rcParams['axes.linewidth'] = 2
plt.rcParams['axes.titlesize'] = 52
plt.rcParams['axes.labelsize'] = 52
plt.rcParams['lines.linewidth'] = 8
plt.rcParams['xtick.labelsize'] = 52
plt.rcParams['ytick.labelsize'] = 52
plt.rcParams['font.family'] = 'serif'
plt.rcParams['legend.fontsize'] = 42
plt.rcParams['figure.dpi'] = 600

# === 策略缩写映射 ===
policy_display_map = {
    "fcfs": "vLLM",
    "fcfs_mc": "Mooncake",
    "ldf_rej_gate1.4_credit": "Scorpio(Ours)",
    "ssjf_pred": "S3"
}

# === 样式映射 ===
policy_style_map = {
    "Scorpio(Ours)": {'color': '#ec6446', 'marker': 's', 'linestyle': '-', 'zorder': 10},
    "vLLM": {'color': '#f29d46', 'marker': '^', 'linestyle': '--', 'zorder': 8},
    "Mooncake": {'color': '#3f906e', 'marker': 'D', 'linestyle': '--', 'zorder': 6},
    "S3": {'color': '#836ed5', 'marker': 'o', 'linestyle': '-', 'zorder': 4},
}

# === 时间 bucket 间隔 ===
bucket_interval = '10s'

# === 主图绘制 ===
plt.figure()

for filename in sorted(os.listdir(".")):
    if not filename.endswith(".json"):
        continue

    base = filename.split("_qps")[0].replace("r1_", "")
    policy_name = policy_display_map.get(base)
    if policy_name is None:
        continue

    with open(filename, "r") as f:
        data = json.load(f)

    records = data["trace_slo_adherence"]
    df = pd.DataFrame(records, columns=["timestamp", "slo_met"])
    df["timestamp"] = pd.to_datetime(df["timestamp"])
    df.set_index("timestamp", inplace=True)

    bucketed = df.resample(bucket_interval).agg(slo_met_count=("slo_met", "sum")).fillna(0)
    bucketed["cumulative_slo_met"] = bucketed["slo_met_count"].cumsum()
    relative_minutes = (bucketed.index - bucketed.index.min()).total_seconds() / 60

    style = policy_style_map[policy_name]
    plt.plot(
        relative_minutes,
        bucketed["cumulative_slo_met"],
        label=policy_name,
        color=style['color'],
        marker=style['marker'],
        linestyle=style['linestyle'],
        linewidth=8,
        markersize=32,
        markevery=10,  # 每隔10个点画一个marker，可根据曲线长度调整
        zorder=style['zorder']
    )
    from matplotlib.ticker import FuncFormatter
    def thousands_formatter(x, pos):
        return f'{int(x/1000)}k' if x >= 1000 else str(int(x))

    plt.gca().yaxis.set_major_formatter(FuncFormatter(thousands_formatter))


# === 主图保存 ===
plt.xlabel("Timeline (min)")
plt.ylabel("Cum. # SLO Met")
plt.grid(True)
plt.tight_layout()
plt.savefig("slo_met_over_time_cdf.pdf", dpi=600, bbox_inches='tight', format='pdf')
plt.close()

# === 单独绘制 legend 图 ===
labels = ['Scorpio(Ours)', 'vLLM', 'Mooncake', 'S3']
colors = ['#ec6446', '#f29d46', '#3f906e', '#836ed5']
markers = ['s', '^', 'D', 'o']
linestyles = ['-', '--', '--', '-']
linewidth = 6
legend_fontsize = 52

fig, ax = plt.subplots(figsize=(25, 1))
handles = [
    mlines.Line2D(
        [], [], color=colors[i], marker=markers[i], linestyle=linestyles[i],
        linewidth=linewidth, label=label, markersize=42
    )
    for i, label in enumerate(labels)
]

ax.legend(handles=handles, loc='center', fontsize=legend_fontsize, ncol=len(labels), frameon=False)
ax.axis('off')
plt.savefig("slo_goodput_legend.pdf", dpi=600, format="pdf", bbox_inches="tight")
plt.close()


In [2]:
import os
import json
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.lines as mlines

# === 样式设置 ===
plt.rcParams['figure.figsize'] = (10, 10)
plt.rcParams['axes.linewidth'] = 2
plt.rcParams['axes.titlesize'] = 52
plt.rcParams['axes.labelsize'] = 52
plt.rcParams['lines.linewidth'] = 8
plt.rcParams['xtick.labelsize'] = 52
plt.rcParams['ytick.labelsize'] = 52
plt.rcParams['font.family'] = 'serif'
plt.rcParams['legend.fontsize'] = 42
plt.rcParams['figure.dpi'] = 600

# === 策略缩写映射 ===
policy_display_map = {
    "fcfs": "vLLM",
    "fcfs_mc": "Mooncake",
    "ldf_rej_gate1.4_credit": "Scorpio(Ours)",
    "ssjf_pred": "S3"
}

# === 样式映射 ===
policy_style_map = {
    "Scorpio(Ours)": {'color': '#ec6446', 'marker': 's', 'linestyle': '-', 'zorder': 10},
    "vLLM": {'color': '#f29d46', 'marker': '^', 'linestyle': '--', 'zorder': 8},
    "Mooncake": {'color': '#3f906e', 'marker': 'D', 'linestyle': '--', 'zorder': 6},
    "S3": {'color': '#836ed5', 'marker': 'o', 'linestyle': '-', 'zorder': 4},
}

# === 时间 bucket 间隔 ===
bucket_interval = '10s'

# === 主图绘制 ===
plt.figure()

recorded_total = False
total_relative_minutes = None
total_cumulative_count = None

for filename in sorted(os.listdir(".")):
    if not filename.endswith(".json"):
        continue

    base = filename.split("_qps")[0].replace("r1_", "")
    policy_name = policy_display_map.get(base)
    if policy_name is None:
        continue

    with open(filename, "r") as f:
        data = json.load(f)

    records = data["trace_slo_adherence"]
    df = pd.DataFrame(records, columns=["timestamp", "slo_met"])
    df["timestamp"] = pd.to_datetime(df["timestamp"])
    df.set_index("timestamp", inplace=True)

    # 提取该策略的 met 曲线
    bucketed = df.resample(bucket_interval).agg(slo_met_count=("slo_met", "sum")).fillna(0)
    bucketed["cumulative_slo_met"] = bucketed["slo_met_count"].cumsum()
    relative_minutes = (bucketed.index - bucketed.index.min()).total_seconds() / 60

    # === 记录一次 total（met + not met） ===
    if not recorded_total:
        total_requests_per_bucket = df.resample(bucket_interval).size().fillna(0)
        total_cumulative_count = total_requests_per_bucket.cumsum()
        total_relative_minutes = (total_requests_per_bucket.index - total_requests_per_bucket.index.min()).total_seconds() / 60
        recorded_total = True

    # === 策略曲线绘图 ===
    style = policy_style_map[policy_name]
    plt.plot(
        relative_minutes,
        bucketed["cumulative_slo_met"],
        label=policy_name,
        color=style['color'],
        marker=style['marker'],
        linestyle=style['linestyle'],
        linewidth=8,
        markersize=32,
        markevery=10,
        zorder=style['zorder']
    )    
    from matplotlib.ticker import FuncFormatter
    def thousands_formatter(x, pos):
        return f'{int(x/1000)}k' if x >= 1000 else str(int(x))

    plt.gca().yaxis.set_major_formatter(FuncFormatter(thousands_formatter))

# # === 添加 total 曲线 ===
# plt.plot(
#     total_relative_minutes,
#     total_cumulative_count,
#     label="Total",
#     color="black",
#     linestyle=":",
#     linewidth=6,
#     zorder=1
# )

# === 主图保存 ===
plt.xlabel("Timeline (min)")
plt.ylabel("Cum. # SLO Met")
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.savefig("slo_met_over_time_cdf.pdf", dpi=600, bbox_inches='tight', format='pdf')
plt.close()

# === 单独绘制 legend 图 ===
labels = ['Scorpio(Ours)', 'vLLM', 'Mooncake', 'S3', 'Total']
colors = ['#ec6446', '#f29d46', '#3f906e', '#836ed5', 'black']
markers = ['s', '^', 'D', 'o', None]
linestyles = ['-', '--', '--', '-', ':']
linewidth = 6
legend_fontsize = 52

fig, ax = plt.subplots(figsize=(30, 1))
handles = [
    mlines.Line2D(
        [], [], color=colors[i], marker=markers[i], linestyle=linestyles[i],
        linewidth=linewidth, label=label, markersize=42 if markers[i] else 0
    )
    for i, label in enumerate(labels)
]

ax.legend(handles=handles, loc='center', fontsize=legend_fontsize, ncol=len(labels), frameon=False)
ax.axis('off')
plt.savefig("slo_goodput_legend.pdf", dpi=600, format="pdf", bbox_inches="tight")
plt.close()
